In [3]:
import yfinance as yf
import pandas as pd
import datetime
import urllib.request  # Importiert, um die Anfrage zu ändern

# --- Schritt 1: Ticker-Liste der S&P 500 von Wikipedia holen ---

print("Lade S&P 500 Ticker-Liste von Wikipedia...")

wiki_url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

# --- KORREKTUR FÜR HTTP ERROR 403 ---
# Wir müssen einen User-Agent-Header hinzufügen, um wie ein Browser auszusehen
req = urllib.request.Request(
    wiki_url,
    headers={'User-Agent': 'Mozilla/5.0'}
)
# Wir öffnen die URL mit urllib...
html_content = urllib.request.urlopen(req)
# ...und übergeben den Inhalt an pandas
tables = pd.read_html(html_content)
# -------------------------------------

sp500_table = tables[0]

# Die Spalte 'Symbol' enthält die Ticker
tickers = sp500_table['Symbol'].tolist()

# WICHTIGE BEREINIGUNG:
# Wikipedia verwendet Ticker wie "BRK.B", yfinance erwartet "BRK-B".
tickers = [ticker.replace('.', '-') for ticker in tickers]

# Optional: Den S&P 500 Index selbst hinzufügen
tickers.append('^GSPC')

print(f"{len(tickers)} Ticker-Symbole gefunden.")


# --- Schritt 2: Daten für 20 Jahre mit yfinance herunterladen ---

# End-Datum (heute)
end_date = datetime.date.today()

# Start-Datum (vor ca. 20 Jahren)
start_date = end_date - datetime.timedelta(days=21 * 365.25)

print(f"Starte Download für 20 Jahre: {start_date} bis {end_date}...")

try:
    data = yf.download(tickers, start=start_date, end=end_date)

    print("--- Download abgeschlossen ---")

    # Zeige das Ergebnis
    print("Struktur der Daten (MultiIndex-DataFrame):")
    print(data.head())

    # WICHTIGER HINWEIS:
    print("\nHinweis: 'NaN'-Werte sind normal.")
    print("Unternehmen, die vor 20 Jahren noch nicht an der Börse waren,")
    print("haben 'NaN' an den Tagen, an denen sie noch nicht gelistet waren.")

except Exception as e:
    print(f"Ein Fehler ist aufgetreten: {e}")
    print("Einige Ticker-Symbole konnten möglicherweise nicht heruntergeladen werden.")


# Um z.B. nur die 'Adj Close'-Preise zu sehen:
# adj_close_data = data['Adj Close']
# print(adj_close_data.head())

Lade S&P 500 Ticker-Liste von Wikipedia...
504 Ticker-Symbole gefunden.
Starte Download für 20 Jahre: 2004-11-09 bis 2025-11-09...


/tmp/ipython-input-4228138023.py:50: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=start_date, end=end_date)
[*********************100%***********************]  504 of 504 completed


--- Download abgeschlossen ---
Struktur der Daten (MultiIndex-DataFrame):
Price           Close                                                      \
Ticker              A      AAPL ABBV ABNB        ABT      ACGL        ACN   
Date                                                                        
2004-11-09  15.305226  0.811294  NaN  NaN  13.328122  4.065620  17.215620   
2004-11-10  14.999844  0.821801  NaN  NaN  13.342935  4.115278  16.915066   
2004-11-11  15.377092  0.830056  NaN  NaN  13.416976  4.075129  16.984957   
2004-11-12  13.520814  0.833058  NaN  NaN  13.085247  4.055053  17.131746   
2004-11-15  13.646564  0.829156  NaN  NaN  13.177066  4.053997  17.607052   

Price                                        ...   Volume                     \
Ticker           ADBE        ADI        ADM  ...     WYNN      XEL       XOM   
Date                                         ...                               
2004-11-09  28.683363  24.376587  12.079054  ...  4469600   996600  1

In [4]:
from google.colab import drive
import os

# --- Schritt 3: Google Drive verbinden ---
print("\nVerbinde Google Drive...")
try:
    drive.mount('/content/drive')
    print("Google Drive erfolgreich verbunden.")
except Exception as e:
    print(f"Fehler beim Verbinden mit Google Drive: {e}")

# --- Schritt 4: DataFrame als CSV-Datei speichern ---

# Definieren Sie den Speicherort in Ihrem Google Drive
# '/content/drive/MyDrive/' ist der Standardpfad zu Ihrem Hauptordner
save_path = '/content/drive/MyDrive/sp500_20_years_data.csv'

print(f"Speichere DataFrame als CSV unter: {save_path}")

try:
    # Überprüfen, ob 'data' existiert (falls der Download-Block fehlschlug)
    if 'data' in locals() and isinstance(data, pd.DataFrame):
        data.to_csv(save_path)
        print("--- Speichern erfolgreich abgeschlossen ---")
    else:
        print("Fehler: Der DataFrame 'data' wurde nicht gefunden oder ist leer.")

except Exception as e:
    print(f"Fehler beim Speichern der Datei: {e}")


Verbinde Google Drive...
Mounted at /content/drive
Google Drive erfolgreich verbunden.
Speichere DataFrame als CSV unter: /content/drive/MyDrive/sp500_20_years_data.csv
--- Speichern erfolgreich abgeschlossen ---
